In [46]:
import numpy as np
import pandas as pd
import time
import pyodbc

In [47]:
taglistfile = 'C:\\Users\meclijda\Documents\TagList.csv'
dfTags = pd.read_csv(taglistfile,";",encoding = "ISO-8859-1")
tagList = dfTags['Tag'].tolist()
print(str(len(tagList))+ ' tags in the taglist.')
#print(dfTags['Tag'].head())
#print(list(set([x for x in tagList if tagList.count(x) > 1])))

63 tags in the taglist.


In [48]:
# read IP21 tag data
strAsOfDate = '01-SEP-15 00:00:00.0'
#strAsOfDate = '01-JAN-14 00:00:00.0'
#strAsOfDate = '24-AUG-15 11:40:00.0'
strUpToDate = '19-JAN-17 00:00:00.0'
pattern = '%d-%b-%y %H:%M:%S.0'
#epochstart = int(time.mktime(time.strptime(strAsOfDate, pattern)))

start_time = time.time()

#strConnect = 'DSN=PO_ODBC;MAXROWS=1000000'
strConnect = "Driver={AspenTech SQLplus};ADS=PO;TIMEOUT=;MAXROWS=10000000;TABLE=;CHARINT=Y;"\
    "CHARFLOAT=N;CHARTIME=N;READONLY=Y;ALLFIELDS=N;ROWID=N;CONVERTERRORS=Y;CHARISNULL=Y;TIBCO=N;"

cnn = pyodbc.connect(strConnect)

#print(tagList[0])
df = pd.DataFrame()
for i, strTagName in enumerate(tagList):
    ts = time.time()
    print('{0:2}'.format(i) + ': getting data from tag ' + strTagName + ' ...')
    sql = "select TS_START, AVG from aggregates "\
        "where name = '" + strTagName + "' "\
        "and TS between '" + strAsOfDate + "' and '" + strUpToDate + "' "\
        "and period = 0:05"
    #print(sql)
    data1 = pd.read_sql(sql, cnn)
    data1['TS_START'] = pd.to_datetime(data1['TS_START'],utc=True)
    #data1['TS_START'] = data1['TS_START'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
    data1.set_index('TS_START',inplace=True)
    data1.columns = [strTagName]
    #data1.to_csv(strTagName.replace(":","_") + '.csv')
    #data2 = data1.resample('1H', how='mean')
    #print(str(data1.count())+" - "+str(data1.mean()))
    df = pd.concat([df, data1], axis=1)
    print('    data loaded in %4.1f seconds' % (time.time()-ts))

cnn.close
elapsed_time = time.time() - start_time
print('Total elapsed time: %5.1f seconds.' % elapsed_time)
#print(df.describe())

 0: getting data from tag POARFQCA-17901:me ...
    data loaded in 62.8 seconds
 1: getting data from tag POARAIA-17903_1a:av ...
    data loaded in 62.3 seconds
 2: getting data from tag POARAIA-17903_3a:av ...
    data loaded in 66.0 seconds
 3: getting data from tag POARTC-17902:me ...
    data loaded in 64.9 seconds
 4: getting data from tag POARAIA-17903_2ab:av ...
    data loaded in 65.6 seconds
 5: getting data from tag POARTI-17912:av ...
    data loaded in 66.6 seconds
 6: getting data from tag POARFCA-17907:me ...
    data loaded in 82.1 seconds
 7: getting data from tag POARTC-17923:me ...
    data loaded in 83.6 seconds
 8: getting data from tag POARTCA-17931:me ...
    data loaded in 78.4 seconds
 9: getting data from tag POARFIA-17904:av ...
    data loaded in 84.5 seconds
10: getting data from tag POARTCA-17922:me ...
    data loaded in 78.1 seconds
11: getting data from tag POARFC-17902:me ...
    data loaded in 88.4 seconds
12: getting data from tag POARTI-17947:av ...

In [49]:
start_time = time.time()
df.to_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total writing time to pickle: %5.1f seconds.' % elapsed_time)

Total writing time to pickle:   0.5 seconds.


In [50]:
start_time = time.time()
df1 = pd.read_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total loading time from pickle: %5.1f seconds.' % elapsed_time)
df1.describe()

Total loading time from pickle:   0.2 seconds.


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,...,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000
mean,23.810105,70.194453,67.033263,95.132076,44.261544,103.511350,3.110707,82.315607,198.469157,132.883182,...,59.071212,17.260786,-63.005108,1.335870,1.520035,205.258451,182.846049,183.863248,165.901360,16.673357
std,2.656670,10.207546,10.248289,1.474526,48.715853,1.215392,0.188184,2.127085,1.569673,22.276006,...,2.892020,2.168786,2.550798,0.287054,0.583993,1.667522,1.462868,1.581137,2.041106,1.952952
min,3.468945,0.000000,0.000000,0.000000,0.000000,96.731805,0.000000,0.000000,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.285192,69.841845,66.244416,94.478933,0.000000,102.763313,3.003630,81.087655,196.733076,132.115791,...,57.435870,17.034638,-65.003986,1.276578,0.827707,203.686828,181.995616,182.922748,164.512036,16.253154
50%,24.610005,71.389814,69.173932,94.522833,0.000000,103.333809,3.102156,81.968798,199.195459,139.826584,...,58.608063,17.844036,-63.076923,1.376249,1.907663,204.959298,182.776285,183.829414,165.704273,17.332788
75%,25.417495,73.344342,70.876237,96.004692,97.873289,104.210291,3.201202,82.092241,199.499731,142.993805,...,60.623924,18.263703,-61.993337,1.477681,2.026987,206.931702,183.481436,184.544172,167.394755,17.879447
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.568412,...,157.287056,23.000000,0.000000,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [51]:
df.describe()

,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,...,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000
mean,23.810105,70.194453,67.033263,95.132076,44.261544,103.511350,3.110707,82.315607,198.469157,132.883182,...,59.071212,17.260786,-63.005108,1.335870,1.520035,205.258451,182.846049,183.863248,165.901360,16.673357
std,2.656670,10.207546,10.248289,1.474526,48.715853,1.215392,0.188184,2.127085,1.569673,22.276006,...,2.892020,2.168786,2.550798,0.287054,0.583993,1.667522,1.462868,1.581137,2.041106,1.952952
min,3.468945,0.000000,0.000000,0.000000,0.000000,96.731805,0.000000,0.000000,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.285192,69.841845,66.244416,94.478933,0.000000,102.763313,3.003630,81.087655,196.733076,132.115791,...,57.435870,17.034638,-65.003986,1.276578,0.827707,203.686828,181.995616,182.922748,164.512036,16.253154
50%,24.610005,71.389814,69.173932,94.522833,0.000000,103.333809,3.102156,81.968798,199.195459,139.826584,...,58.608063,17.844036,-63.076923,1.376249,1.907663,204.959298,182.776285,183.829414,165.704273,17.332788
75%,25.417495,73.344342,70.876237,96.004692,97.873289,104.210291,3.201202,82.092241,199.499731,142.993805,...,60.623924,18.263703,-61.993337,1.477681,2.026987,206.931702,183.481436,184.544172,167.394755,17.879447
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.568412,...,157.287056,23.000000,0.000000,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [52]:
start_time = time.time()
df.to_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total writing time to hdf5: %5.1f seconds.' % elapsed_time)

Total writing time to hdf5:   0.7 seconds.


In [53]:
start_time = time.time()
df2 = pd.read_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total loading time from hdf5: %5.1f seconds.' % elapsed_time)
df2.describe()

Total loading time from hdf5:   0.1 seconds.


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,...,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000,145740.000000
mean,23.810105,70.194453,67.033263,95.132076,44.261544,103.511350,3.110707,82.315607,198.469157,132.883182,...,59.071212,17.260786,-63.005108,1.335870,1.520035,205.258451,182.846049,183.863248,165.901360,16.673357
std,2.656670,10.207546,10.248289,1.474526,48.715853,1.215392,0.188184,2.127085,1.569673,22.276006,...,2.892020,2.168786,2.550798,0.287054,0.583993,1.667522,1.462868,1.581137,2.041106,1.952952
min,3.468945,0.000000,0.000000,0.000000,0.000000,96.731805,0.000000,0.000000,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.285192,69.841845,66.244416,94.478933,0.000000,102.763313,3.003630,81.087655,196.733076,132.115791,...,57.435870,17.034638,-65.003986,1.276578,0.827707,203.686828,181.995616,182.922748,164.512036,16.253154
50%,24.610005,71.389814,69.173932,94.522833,0.000000,103.333809,3.102156,81.968798,199.195459,139.826584,...,58.608063,17.844036,-63.076923,1.376249,1.907663,204.959298,182.776285,183.829414,165.704273,17.332788
75%,25.417495,73.344342,70.876237,96.004692,97.873289,104.210291,3.201202,82.092241,199.499731,142.993805,...,60.623924,18.263703,-61.993337,1.477681,2.026987,206.931702,183.481436,184.544172,167.394755,17.879447
max,28.730017,101.736916,78.562834,130.613648,100.000000,113.543055,3.512904,90.413389,203.653258,148.568412,...,157.287056,23.000000,0.000000,4.140800,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [54]:
start_time = time.time()
df.to_csv('rawdata.csv')
elapsed_time = time.time() - start_time
print('Total writing time to csv: %5.1f seconds.' % elapsed_time)

Total writing time to csv:  44.0 seconds.
